Todos:
* generate test data set
    - number of clusters
    - space between clusters (from this the max pivot deviation is calculated)
    - values per cluster (also random range possible, then the start and end e.g. 10-42 should be possible -make adapted function for this- )
* is the above okay since it is a unique distribution and cluster centers might vary... in the beginning yes, it is one of many tests and this one tests specifically how ant clust performs on many uniformly distributed clusters. 

Idea:
* make function that generates the set
* each cluster has a pivot element from which single data points deviate.
* the pivot is a unique cluster id (no need to label the clusters)
    
    
    

In [1]:
# make AntClus dir known
import sys

sys.path.append("../AntClust")

# generally used
import random as rng

import numpy as np
from AntClust import AntClust

# AntClust
from distance_classes import similarity_1d
from rules import labroche_rules

In [2]:
def uniform_cluster_data_random(
    num_clusters,
    cluster_width,
    values_per_cluster_min=0,
    values_per_cluster_max=100,
    rng_seed=1,
):
    """
    Makes a series of numbers that can be clustered together.
    Each cluster is getting a pivot element, starting by one and counts to
    num_clusters. From this a random deviation is generated which will be
    the new data point. If the deviation is to larg there will be a string
    of numbers with no space in between and as such no clusters.
    cluster_width: how far can the data point be from its pivot,
    only values <0.25 makes sense.
    """
    # variables
    data = []
    labels = []

    rng.seed(rng_seed)
    rand_min = -cluster_width
    rand_max = cluster_width
    rand_range = rand_max - rand_min

    # make data for each cluster pivot
    for c_pivot in range(1, num_clusters + 1):
        c_data = []
        c_labels = []

        # how many values to generate?
        values_per_cluster = rng.randin(values_per_cluster_min, values_per_cluster_max)
        # generate data points
        for n in range(0, values_per_cluster):
            # calc pivot deviation and add it
            dp = [c_pivot + (rand_min + rng.random() * rand_range)]

            # check if exist and if create new
            while dp in c_data:
                dp = [c_pivot + (rand_min + rng.random() * rand_range)]

            # add it with the labe to the cluster data
            c_data.append(dp)
            c_labels.append(c_pivot - 1)

        # add the cluster data to overall data
        data = data + c_data
        labels = labels + c_labels

    return data, labels


def uniform_cluster_data(num_clusters, cluster_width, values_per_cluster, rng_seed=1):
    """
    Makes a series of numbers that can be clustered together.
    Each cluster is getting a pivot element, starting by one and counts to
    num_clusters. From this a random deviation is generated which will be
    the new data point. If the deviation is to larg there will be a string
    of numbers with no space in between and as such no clusters.
    cluster_width: how far can the data point be from its pivot,
    only values <0.25 makes sense.
    """
    # variables
    data = []
    labels = []

    rng.seed(rng_seed)
    rand_min = -cluster_width
    rand_max = cluster_width
    rand_range = rand_max - rand_min

    # make data for each cluster pivot
    for c_pivot in range(1, num_clusters + 1):
        c_data = []
        c_labels = []

        # generate data points
        for n in range(0, values_per_cluster):
            # calc pivot deviation and add it
            dp = [c_pivot + (rand_min + rng.random() * rand_range)]

            # check if exist and if create new
            while dp in c_data:
                dp = [c_pivot + (rand_min + rng.random() * rand_range)]

            # add it with the labe to the cluster data
            c_data.append(dp)
            c_labels.append(c_pivot - 1)

        # add the cluster data to overall data
        data = data + c_data
        labels = labels + c_labels

    return data, labels


def make_cluster_mask(clust_labels):
    """
    This function will make a cluster mask which will
    hold the start and end index from each cluster.
    """
    clust_mask = []
    # get current clust
    current_clust_label = clust_labels[0]
    current_clust_start = 0

    # loop through all clusts
    for i in range(len(clust_labels)):
        if clust_labels[i] != current_clust_label:
            clust_mask.append([current_clust_start, i - 1])
            current_clust_label = clust_labels[i]
            current_clust_start = i
    # append the final mask
    clust_mask.append([current_clust_start, i])
    
    return clust_mask


def error_cluster_mask(cluster_result, cluster_mask, print_error_per_cluster=False):
    """
    Will test the cluster error based on the cluster mask approach.
    For every cluster in the mask it is checked how many clusters where found
    in the respective area. It is determined based on the highest representation
    number how many how many missclassified data points exist.
    It is assumed that the cluster labels are ascendendly ordered and no number
    in that row is missing, e.g.:
    correct : [0,0,0,0,1,1,1,2,2,...]
    false   : [0,0,0,3,3,2,2,....]
    """
    # vars
    cluster_errors = []
    already_used_labels = []

    # find the most significant number inside the clust range
    for c_range in cluster_mask:
        label_dict = {}

        # count numbers inside the cluster range
        for i in range(c_range[0], c_range[1] + 1):
            if cluster_result[i] not in label_dict:
                label_dict[cluster_result[i]] = 1
            else:
                label_dict[cluster_result[i]] += 1
        # find most dominant one, going to assumt this is the cluster label.
        # If two labels are equally dominant use the first occured
        max_index = np.where(
            list(label_dict.values()) == np.max(list(label_dict.values()))
        )[0][0]
        max_key = list(label_dict.keys())[max_index]
        
        # save nums with key and occurence and clust range index
        # count highest
        # if used check the occurences
        # if occurence here more then reassign and remove from the old one 
        #     and find a new one for the old one by counting and us one that es not already used
        #     If no is found assign no cluster label
        # else (occurence less) go through all others until found one that is not used
        #     if all are used assign no label
        # save the labels in a list where [clust_range, label]
        # then you can go throug that list and count the errors (full error if label -1)

        # count for errors
        # now check how many of the same label are inside the cluster which tells
        # the ones that are correctly clustered.
        num_correct = 0
        for i in range(c_range[0], c_range[1] + 1):
            if cluster_result[i] == max_key:
                num_correct += 1

        # calc and append error
        cluster_errors.append(1 - (num_correct / (c_range[1] - c_range[0] + 1)))

    # return overall error
    if print_error_per_cluster:
        print(cluster_errors)
    return sum(cluster_errors) / len(cluster_errors)


# TODO:
# - run with different seeds and take the mean error
def run_benchmark():
    # generate two for loops that generate the data and then run antclust
    # and save the error to a 2d list of lists

    return []

In [4]:
# ------------------
# data
# ------------------
num_clusters = 5
cluster_width = 0.1  # this defines how hard the clustering task is?
values_per_cluster = 100
data, labels = uniform_cluster_data(
    num_clusters, cluster_width, values_per_cluster, rng_seed=3
)
data = np.array(data)
labels = np.array(labels)

cluster_mask = make_cluster_mask(labels)
error_cluster_mask(labels, cluster_mask)
cluster_mask

[[0, 99], [100, 199], [200, 299], [300, 399], [400, 499]]

In [30]:
# ------------------
# data
# ------------------
num_clusters = 30
cluster_width = 0.1  # this defines how hard the clustering task is?
values_per_cluster = 10
data, labels = uniform_cluster_data(
    num_clusters, cluster_width, values_per_cluster, rng_seed=3
)
data = np.array(data)
labels = np.array(labels)

# ------------------
# AntClust
# ------------------
# similarity function
f_sim = [similarity_1d(data.min(), data.max())]

# rules
rules = labroche_rules()

# AntClust
ant_clust = AntClust(data, f_sim, rules, alpha_ant_meeting_iterations=150)

# find clusters
ant_clust.find_clusters()

# get the clustering result
clusters_found = ant_clust.get_clusters()

# ------------------
# performance
# ------------------
# how many labels differ?
num_missclassified = len(np.where(clusters_found != labels)[0])
error = num_missclassified / len(data)
print(f"label error: {error}")

# error with cluster mask approach
cluster_mask = make_cluster_mask(labels)
error = error_cluster_mask(clusters_found, cluster_mask, print_error_per_cluster=True)
print(f'cluster mask error: {error}')

# labels found
print(f'clusters found ids \n')
clusters_found

AntClust: phase 1 of 3 -> meeting ants
left meetings  22500 / 22500
left meetings  20250 / 22500
left meetings  18000 / 22500
left meetings  15750 / 22500
left meetings  13500 / 22500
left meetings  11250 / 22500
left meetings  9000 / 22500
left meetings  6750 / 22500
left meetings  4500 / 22500
left meetings  2250 / 22500
AntClust: phase 2 of 3 -> shrink nests
AntClust: phase 3 of 3 -> reassign ants
label error: 0.9666666666666667
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09999999999999998, 0.4, 0.4, 0.4, 0.6, 0.4, 0.6, 0.5, 0.0, 0.0, 0.09999999999999998, 0.0, 0.0, 0.4, 0.5, 0.19999999999999996, 0.19999999999999996, 0.09999999999999998, 0.09999999999999998, 0.0, 0.0]
cluster mask error: 0.16666666666666666
clusters found ids 



array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 1, 4, 1, 0,
       1, 0, 3, 3, 3, 1, 0, 1, 1, 1, 4, 1, 4, 1, 1, 4, 1, 3, 3, 1, 4, 4,
       3, 4, 3, 1, 4, 3, 1, 3, 3, 3, 3, 4, 1, 1, 4, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       5, 5, 3, 5, 5, 3, 3, 3, 3, 3, 3, 5, 5, 3, 3, 3, 5, 5, 5, 3, 3, 3,
       3, 3, 5, 5, 3, 3, 3, 3, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [31]:
labels

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22,
       22, 22, 22, 22, 22

In [32]:
clusters_found

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 1, 4, 1, 0,
       1, 0, 3, 3, 3, 1, 0, 1, 1, 1, 4, 1, 4, 1, 1, 4, 1, 3, 3, 1, 4, 4,
       3, 4, 3, 1, 4, 3, 1, 3, 3, 3, 3, 4, 1, 1, 4, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       5, 5, 3, 5, 5, 3, 3, 3, 3, 3, 3, 5, 5, 3, 3, 3, 5, 5, 5, 3, 3, 3,
       3, 3, 5, 5, 3, 3, 3, 3, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

# scikit tests

In [39]:
labels

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22,
       22, 22, 22, 22, 22

In [40]:
clusters_found

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 1, 4, 1, 0,
       1, 0, 3, 3, 3, 1, 0, 1, 1, 1, 4, 1, 4, 1, 1, 4, 1, 3, 3, 1, 4, 4,
       3, 4, 3, 1, 4, 3, 1, 3, 3, 3, 3, 4, 1, 1, 4, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       5, 5, 3, 5, 5, 3, 3, 3, 3, 3, 3, 5, 5, 3, 3, 3, 5, 5, 5, 3, 3, 3,
       3, 3, 5, 5, 3, 3, 3, 3, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [41]:
from sklearn.metrics import rand_score
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import mutual_info_score


# RI = (number of agreeing pairs) / (number of pairs)
ris = rand_score(labels, clusters_found)
print(ris)

# ARI = (RI - Expected_RI) / (max(RI) - Expected_RI)
ari = adjusted_rand_score(labels, clusters_found)
print(ari)

# 
#ss = silhouette_score(labels, clusters_found)

#print(ss)


# 
#dbs = davies_bouldin_score(labels, clusters_found)
#print(dbs)


  
mis = mutual_info_score(labels, clusters_found)
print(mis)

0.7472017837235229
0.10545372984668971
1.0815245607018105


In [ ]:
cluster_mask(labels)

In [ ]:
data

In [ ]:
a = [[1], [2]]
[1] in a

In [ ]:
import random as rng

In [ ]:
min = -0.25
max = 0.25

r = max - min


min + rng.random() * r

In [ ]:
min = 0.25
max = 0.5

r = max - min

min + 0.99999 * r

In [ ]:
rng.randint(10, 100)

In [ ]:
0.9999 * r